In [1]:
import pandas as pd
import numpy as np

In [26]:
seeds = pd.read_csv('../data/NCAA/MNCAATourneySeeds.csv')
results = pd.read_csv('../data/NCAA/MNCAATourneyCompactResults.csv')

In [27]:
seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [28]:
results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [29]:
seeds['Seed'] = seeds['Seed'].str[1:].str[:2].astype(int)

In [30]:
results = results[['Season', 'WTeamID', 'WScore', 'LTeamID', 'LScore']]

In [31]:
results.columns = ['Season', 'T1TeamID', 'T1Score', 'T2TeamID', 'T2Score']

In [32]:
results.columns

Index(['Season', 'T1TeamID', 'T1Score', 'T2TeamID', 'T2Score'], dtype='object')

In [33]:
results_swap = results[['Season', 'T2TeamID', 'T2Score', 'T1TeamID', 'T1Score']]

In [35]:
results_swap.columns = results.columns
#= ['Season', 'T1TeamID', 'T1Score', 'T2TeamID', 'T2Score']

In [36]:
results['Result'] = 1

In [37]:
results_swap['Result'] = 0

In [39]:
results

,Season,T1TeamID,T1Score,T2TeamID,T2Score,Result
0,1985,1116,63,1234,54,1
1,1985,1120,59,1345,58,1
2,1985,1207,68,1250,43,1
3,1985,1229,58,1425,55,1
4,1985,1242,49,1325,38,1
...,...,...,...,...,...,...
2246,2019,1120,77,1246,71,1
2247,2019,1277,68,1181,67,1
2248,2019,1403,61,1277,51,1
2249,2019,1438,63,1120,62,1


In [41]:
game_data = pd.concat([results, results_swap])

In [43]:
game_data = game_data.merge(seeds, how='left', left_on=['Season', 'T1TeamID'], right_on=['Season', 'TeamID'])

In [44]:
game_data.rename({'Seed': 'T1Seed'}, axis=1, inplace=True)

In [45]:
game_data.drop('TeamID', axis=1, inplace=True)

In [47]:
game_data = game_data.merge(seeds, how='left', left_on=['Season', 'T2TeamID'], right_on=['Season', 'TeamID'])

In [48]:
game_data.rename({'Seed': 'T2Seed'}, axis=1, inplace=True)
game_data.drop('TeamID', axis=1, inplace=True)

In [49]:
game_data['SeedDiff'] = game_data['T1Seed']  - game_data['T2Seed']
game_data.head()

,Season,T1TeamID,T1Score,T2TeamID,T2Score,Result,T1Seed,T2Seed,SeedDiff
0,1985,1116,63,1234,54,1,9,8,1
1,1985,1120,59,1345,58,1,11,6,5
2,1985,1207,68,1250,43,1,1,16,-15
3,1985,1229,58,1425,55,1,9,8,1
4,1985,1242,49,1325,38,1,3,14,-11


In [50]:
from sklearn.ensemble import RandomForestClassifier

In [51]:
rf = RandomForestClassifier()

In [54]:
X = game_data[['SeedDiff']]
y = game_data['Result']

In [55]:
rf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [56]:
X.shape

(4502, 1)

In [57]:
y.shape

(4502,)

In [58]:
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [59]:
rf.predict(X)

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [60]:
rf.score(X, y)

0.7099067081297201

In [63]:
rf.predict_proba(X)

array([[0.54054751, 0.45945249],
       [0.66839597, 0.33160403],
       [0.00717788, 0.99282212],
       ...,
       [0.46062844, 0.53937156],
       [0.69931882, 0.30068118],
       [0.54384378, 0.45615622]])

In [64]:
season = pd.read_csv('../data/NCAA/MRegularSeasonCompactResults.csv')

In [65]:
season.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [70]:
wins =season.groupby(['Season', 'WTeamID'])['WScore'].count().reset_index(name='Wins')

In [71]:
losses = season.groupby(['Season', 'LTeamID'])['LScore'].count().reset_index(name='Losses')

In [76]:
game_data = game_data.merge(wins, how='left', left_on=['Season', 'T1TeamID'], right_on=['Season', 'WTeamID'])

In [77]:
game_data = game_data.merge(losses, how='left', left_on=['Season', 'T1TeamID'], right_on=['Season', 'LTeamID'])

In [79]:
game_data = game_data.drop(['WTeamID', 'LTeamID'], axis=1)

In [81]:
game_data.rename({'Losses': 'T1Losses', 'Wins': 'T1Wins'}, axis=1, inplace=True)

In [82]:
game_data

,Season,T1TeamID,T1Score,T2TeamID,T2Score,Result,T1Seed,T2Seed,SeedDiff,T1Wins,T1Losses
0,1985,1116,63,1234,54,1,9,8,1,21,12.0
1,1985,1120,59,1345,58,1,11,6,5,18,11.0
2,1985,1207,68,1250,43,1,1,16,-15,25,2.0
3,1985,1229,58,1425,55,1,9,8,1,20,7.0
4,1985,1242,49,1325,38,1,3,14,-11,23,7.0
...,...,...,...,...,...,...,...,...,...,...,...
4497,2019,1246,71,1120,77,0,2,5,-3,27,6.0
4498,2019,1181,67,1277,68,0,1,2,-1,29,5.0
4499,2019,1277,51,1403,61,0,2,3,-1,28,6.0
4500,2019,1120,62,1438,63,0,5,1,4,25,9.0


In [86]:
game_data.drop(['T1Wins', 'T1Losses'], axis=1, inplace=True)

In [88]:
pwd

'C:\\Users\\afs95\\MyPython\\DataScienceGA\\DS_work\\DS_repo\\Lectures\\Unit4\\Class18'

In [92]:
game_data.to_csv('../data/NCAA/game_data.csv', index=False)